In [2]:
import pandas as pd
import numpy as np

In [3]:
list_of_df = []
n_epochs = 20

epoch_string = 'bert-base/20_epochs_baseline/saved_report_1b/bert-base-uncased/1/'

for epoch in range(n_epochs):
    epoch_string += 'epoch_' + str(epoch) + '/'
    epoch_location = epoch_string + 'unformatted_result.tsv'
    print(epoch_location)
    list_of_df.append(pd.read_csv(epoch_location, sep='\t'))

bert-base/20_epochs_baseline/saved_report_1b/bert-base-uncased/1/epoch_0/unformatted_result.tsv
bert-base/20_epochs_baseline/saved_report_1b/bert-base-uncased/1/epoch_0/epoch_1/unformatted_result.tsv
bert-base/20_epochs_baseline/saved_report_1b/bert-base-uncased/1/epoch_0/epoch_1/epoch_2/unformatted_result.tsv
bert-base/20_epochs_baseline/saved_report_1b/bert-base-uncased/1/epoch_0/epoch_1/epoch_2/epoch_3/unformatted_result.tsv
bert-base/20_epochs_baseline/saved_report_1b/bert-base-uncased/1/epoch_0/epoch_1/epoch_2/epoch_3/epoch_4/unformatted_result.tsv
bert-base/20_epochs_baseline/saved_report_1b/bert-base-uncased/1/epoch_0/epoch_1/epoch_2/epoch_3/epoch_4/epoch_5/unformatted_result.tsv
bert-base/20_epochs_baseline/saved_report_1b/bert-base-uncased/1/epoch_0/epoch_1/epoch_2/epoch_3/epoch_4/epoch_5/epoch_6/unformatted_result.tsv
bert-base/20_epochs_baseline/saved_report_1b/bert-base-uncased/1/epoch_0/epoch_1/epoch_2/epoch_3/epoch_4/epoch_5/epoch_6/epoch_7/unformatted_result.tsv
bert-bas

In [4]:
original_df = pd.read_csv('Datasets/Subtask_1b/training/dev.tsv', sep='\t')
original_df

,tweet_id,class,begin,end,span,tweet
0,SMM4H2022ykI8vN7jZYnV57AM,ADE,119,125,nerves,@USER_________ i found the humira to fix all m...
1,SMM4H2022ykI8vN7jZYnV57AM,ADE,126,139,muscle spasms,@USER_________ i found the humira to fix all m...
2,SMM4H2022uCZV2SRsCe4vzjFm,ADE,61,68,gaining,@USER__________ have to go to a doc now to see...
3,SMM4H2022uCZV2SRsCe4vzjFm,ADE,91,110,gain like 50 pounds,@USER__________ have to go to a doc now to see...
4,SMM4H20229Aha6m4XERqYdFWf,ADE,118,134,frontal headache,06.30 day 14 Rivaroxaban diary. Thanks to para...
...,...,...,...,...,...,...
82,SMM4H2022cjwbGQbnkpVjjJzR,ADE,27,39,cotton mouth,"Fucking Vyvanse, giving me cotton mouth. UGH."
83,SMM4H2022lX1A8RVuySkDzAB2,ADE,84,104,couldn't fall asleep,between the fucking redbull and vyvanse i popp...
84,SMM4H2022qZC2BPG2BW7UC175,ADE,45,48,OCD,"rt @USER_______: vyvanse, commonly known as oc..."
85,SMM4H20226b5WyZPTAJ7qL3dE,ADE,63,71,addicted,rt @USER_______: @USE when are you going to do...


In [63]:
best_results = []

for index, row in original_df.iterrows():
    # getting the original values in the tweet (to minimize shuffling)
    original_tweet_id = row['tweet_id']
    original_begin = row['begin']
    original_end = row['end']
    
    # go through every epoch's row of data 
    specific_row_result = []
    for epoch in range(len(list_of_df)): 
        row = list_of_df[epoch].loc[(list_of_df[epoch]['id'] == original_tweet_id) & (list_of_df[epoch]['orig_begin'] == original_begin) & (list_of_df[epoch]['orig_end'] == original_end)]
        # print(row)
        # retrieving the length of the span
        predicted_begin = row['predict_begin'].values[0]
        predicted_end = row['predict_end'].values[0]
        span_length = predicted_end - predicted_begin
        
        if span_length == 0:
            continue

        
        specific_row_result.append((epoch, row['Unnamed: 0'].values[0], span_length))
    
    # sort the specific_row_results by increasing span length
    specific_row_result.sort(key = lambda x : x[2])
    best_results.append(specific_row_result)

# go through every specific_row_results and keeping the median value ONLY
for row in range(len(best_results)):
    
    length_specific_result = len(best_results[row])
    
    if length_specific_result == 0: 
        continue

    median_of_length_specific_result = int(length_specific_result / 2)

    best_results[row] = best_results[row][median_of_length_specific_result]


# appending all the results to create one final result
unformatted_merged_result = pd.DataFrame(columns = ['id', 'text', 'orig_begin', 'orig_end', 'orig_span', 'orig_span_convert', 'predict_span', 'predict_begin', 'predict_end'])

for row in range(len(best_results)):
    if best_results[row] == []:
        # retrieving it from the original dataframe
        original_row_result = original_df.iloc[row]
        tweet_id = original_row_result['tweet_id']
        orig_sentence = original_row_result['tweet']
        orig_begin = original_row_result['begin']
        orig_end = original_row_result['end']
        orig_span = original_row_result['span']
        len_orig_sentence = len(orig_sentence)
        
        # creating a dataframe row and adding the lengths of the original sentences to the result
        
        to_append = pd.DataFrame({'id': [tweet_id], 'text': [orig_sentence], 'orig_begin': [orig_begin], 'orig_end': [orig_end], 'orig_span': [orig_span], 'orig_span_convert': ["No guess"], 'predict_span': [orig_sentence], 'predict_begin': [0], 'predict_end': [len_orig_sentence]})
        unformatted_merged_result = unformatted_merged_result.append(to_append, ignore_index=True)

        continue
    
    original_row_result = list_of_df[best_results[row][0]].iloc[best_results[row][1]]
    tweet_id = original_row_result['id']
    orig_sentence = original_row_result['text']
    orig_begin = original_row_result['orig_begin']
    orig_end = original_row_result['orig_end']
    orig_span = original_row_result['orig_span']
    orig_span_convert = original_row_result['orig_span_convert']
    predict_span = original_row_result['predict_span']
    predict_begin = original_row_result['predict_begin']
    predict_end = original_row_result['predict_end']

    to_append = pd.DataFrame({'id': [tweet_id], 'text': [orig_sentence], 'orig_begin': [orig_begin], 'orig_end': [orig_end], 'orig_span': [orig_span], 'orig_span_convert': [orig_span_convert], 'predict_span': [predict_span], 'predict_begin': [predict_begin], 'predict_end': [predict_end]})
    unformatted_merged_result = unformatted_merged_result.append(to_append, ignore_index=True)

    
print(unformatted_merged_result)


                           id  \
0   SMM4H2022ykI8vN7jZYnV57AM   
1   SMM4H2022ykI8vN7jZYnV57AM   
2   SMM4H2022uCZV2SRsCe4vzjFm   
3   SMM4H2022uCZV2SRsCe4vzjFm   
4   SMM4H20229Aha6m4XERqYdFWf   
..                        ...   
82  SMM4H2022cjwbGQbnkpVjjJzR   
83  SMM4H2022lX1A8RVuySkDzAB2   
84  SMM4H2022qZC2BPG2BW7UC175   
85  SMM4H20226b5WyZPTAJ7qL3dE   
86  SMM4H20220LbDv7WiGBW7nQR4   

                                                 text orig_begin orig_end  \
0   @USER_________ i found the humira to fix all m...        119      125   
1   @USER_________ i found the humira to fix all m...        126      139   
2   @USER__________ have to go to a doc now to see...         61       68   
3   @USER__________ have to go to a doc now to see...         91      110   
4   06.30 day 14 Rivaroxaban diary. Thanks to para...        118      134   
..                                                ...        ...      ...   
82      Fucking Vyvanse, giving me cotton mouth. UGH.         27

In [64]:
# Saving the result
formatted_merged_result = unformatted_merged_result

unformatted_merged_result.to_csv('bert-base/20_epochs_baseline/unformatted_merged_result.tsv', sep='\t', index=False)
formatted_merged_result.to_csv('bert-base/20_epochs_baseline/formatted_merged_reuslt.tsv', sep='\t', index=False, header=False)

